In [1]:
import sys
from pathlib import Path
from importlib import reload

project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

import pandas as pd
import src.model
import src.trainer
import src.visualization

reload(src.model)
reload(src.trainer)
reload(src.visualization)

from src.model import Decoder
from src.trainer import TimeSeriesDataset, TanaForecastTrainer

train_df = pd.read_csv(project_root / 'src' / 'datasets' / 'delhi' / 'DailyDelhiClimateTrain.csv')
test_df = pd.read_csv(project_root / 'src' / 'datasets' / 'delhi' / 'DailyDelhiClimateTest.csv')

train_df.describe()

,meantemp,humidity,wind_speed,meanpressure
count,1462.000000,1462.000000,1462.000000,1462.000000
mean,25.495521,60.771702,6.802209,1011.104548
std,7.348103,16.769652,4.561602,180.231668
min,6.000000,13.428571,0.000000,-3.041667
25%,18.857143,50.375000,3.475000,1001.580357
50%,27.714286,62.625000,6.221667,1008.563492
75%,31.305804,72.218750,9.238235,1014.944901
max,38.714286,100.000000,42.220000,7679.333333


In [2]:
import torch

feature_cols = ['meantemp', 'humidity', 'wind_speed', 'meanpressure']
target_cols = ['meantemp']

train_dataset = TimeSeriesDataset(
    df=train_df,
    context_window=90,
    prediction_length=7,
    feature_columns=feature_cols,
    target_columns=target_cols,
    stride=1,
    normalize=True
)

val_dataset = TimeSeriesDataset(
    df=test_df,
    context_window=90,
    prediction_length=7,
    feature_columns=feature_cols,
    target_columns=target_cols,
    stride=1,
    normalize=True
)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Val dataset size: {len(val_dataset)}")

context, target = train_dataset[0]
print(f"Context shape: {context.shape}")
print(f"Target shape: {target.shape}")


Train dataset size: 1366
Val dataset size: 18
Context shape: torch.Size([4, 90])
Target shape: torch.Size([1, 7])


In [3]:
from src.utils import Loss

model = Decoder(
    context_window=90,
    prediction_length=7,
    d_model=len(feature_cols)
)

trainer = TanaForecastTrainer(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    batch_size=64,
    learning_rate=1e-3,
    num_epochs=50,
    checkpoint_dir=str(project_root / 'checkpoints' / 'delhi'),
    early_stopping_patience=-1,
    loss_fn=Loss.quantile_loss,   # Direct reference
    loss_name='Quantile_0.9',      # Custom name for logging
    loss_kwargs={'q': 0.9}         # Quantile level
)

history = trainer.train()


Training on cpu
Total epochs: 50
Batch size: 64
Train batches: 22
Val batches: 1
------------------------------------------------------------
Epoch 1/50 | Train Loss: 0.176145 | Val Loss: 0.233373 | LR: 9.99e-04 | Time: 0.10s
  → New best model saved (Val Loss: 0.233373)
Epoch 2/50 | Train Loss: 0.076456 | Val Loss: 0.199571 | LR: 9.96e-04 | Time: 0.10s
  → New best model saved (Val Loss: 0.199571)
Epoch 3/50 | Train Loss: 0.066943 | Val Loss: 0.178872 | LR: 9.91e-04 | Time: 0.10s
  → New best model saved (Val Loss: 0.178872)
Epoch 4/50 | Train Loss: 0.060601 | Val Loss: 0.154691 | LR: 9.84e-04 | Time: 0.10s
  → New best model saved (Val Loss: 0.154691)
Epoch 5/50 | Train Loss: 0.056853 | Val Loss: 0.139981 | LR: 9.76e-04 | Time: 0.10s
  → New best model saved (Val Loss: 0.139981)
Epoch 6/50 | Train Loss: 0.056121 | Val Loss: 0.204326 | LR: 9.65e-04 | Time: 0.10s
Epoch 7/50 | Train Loss: 0.053135 | Val Loss: 0.184448 | LR: 9.53e-04 | Time: 0.14s
Epoch 8/50 | Train Loss: 0.050518 | Val 

In [4]:
from src.visualization import plot_training_history

fig = plot_training_history(history)
fig.show()

print(f"\nBest Validation Loss: {min(history['val_loss']):.6f}")
print(f"Final Train Loss: {history['train_loss'][-1]:.6f}")



Best Validation Loss: 0.116791
Final Train Loss: 0.035206


In [5]:
from src.visualization import plot_forecast, compute_metrics, print_metrics

context, target = val_dataset[10]
prediction = trainer.predict(context.unsqueeze(0)).squeeze(0)

fig = plot_forecast(
    context=context,
    target=target,
    prediction=prediction,
    dataset=val_dataset,
    title='Delhi Temperature Forecast (7-day ahead)',
    feature_idx=0,
    feature_name='Temperature (°C)'
)
fig.show()

target_denorm = val_dataset.denormalize(target, is_target=True)
prediction_denorm = val_dataset.denormalize(prediction, is_target=True)

metrics = compute_metrics(target_denorm[0], prediction_denorm[0])
print_metrics(metrics)


MSE:  1.6236
MAE:  1.0568
RMSE: 1.2742
MAPE: 0.0351
SMAPE: 0.0345
